In [1]:
!python --version

Python 3.9.19


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import matplotlib.pyplot as plt
import glob
from tqdm.autonotebook import tqdm
import matplotlib.dates as mdates
from itertools import islice
from collections import defaultdict
from datetime import datetime
import sklearn.metrics
import numpy as np



from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST, TFT
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss
from neuralforecast.losses.numpy import mae, mape, mase, mqloss, mse, quantile_loss, rmse, smape

/tmp/ipykernel_2594428/153247262.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      #examples["country"].append(country)
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      #examples["gen_forecast"].append(sub_df["gen_forecast"][start:context_end + horizon_len].tolist())
      #examples["week_day"].append(sub_df["week_day"][start:context_end + horizon_len].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end + horizon_len)])

    return examples

In [4]:
def get_batched_data_fn_predict(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      #examples["country"].append(country)
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      #examples["gen_forecast"].append(sub_df["gen_forecast"][start:context_end + horizon_len].tolist())
      #examples["week_day"].append(sub_df["week_day"][start:context_end + horizon_len].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end)])

    return examples

In [ ]:
def forecast_building(df, df_predict):

    horizon = 24
    model_patch_tst = PatchTST(h=horizon,
                    input_size=7*horizon,
                    max_steps=100,
                    loss=DistributionLoss(distribution='StudentT', level=[80, 90])
                              )
    
    model_tft = TFT(h=horizon,
                    input_size=7*horizon,
                    max_steps=100,
                    loss=DistributionLoss(distribution='StudentT', level=[80, 90]))
    nf = NeuralForecast(
        models=[model_patch_tst, model_tft],
        freq='H'
    )
    nf.fit(df=df)
    forecasts = nf.predict(df=df_predict)

    res_all = []
    for i in forecasts.index.unique():
        op = df[df.unique_id == i]
        tr = op[op.ds.isin(forecasts[forecasts.index == i]['ds'])][['ds', 'y']]
        tr['y_pred_PatchTST'] = forecasts[forecasts.index == i]['PatchTST-median'].values
        tr['y_pred_TFT'] = forecasts[forecasts.index == i]['TFT-median'].values
        res_all.append(tr)

    res_all_df = pd.concat(res_all).sort_values('ds')
    return res_all_df


In [6]:
def process_building(df): 
    building_name = df.columns[0]
    df.columns = ['y']
    input_data = get_batched_data_fn(df, batch_size=500)
    predict_data = get_batched_data_fn_predict(df, batch_size=500)
    # print(input_data)
    
    windows_all = []
    windows_pred = []
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(input_data['inputs_ts'], 
                                                      input_data['inputs'], 
                                                      input_data['outputs_ts'], 
                                                      input_data['outputs']):
        
        input_df = pd.DataFrame({'ds': inputs_ts, 
                                 'y': inputs})
        
        output_df = pd.DataFrame({'ds': outputs_ts, 
                                 'y': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['unique_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        # combined = combined[combined.y.isnull() == 0]
        # if len(combined) == 192:
        #         windows_all.append(combined)
        # else: continue
        windows_all.append(combined)
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(predict_data['inputs_ts'], 
                                                      predict_data['inputs'], 
                                                      predict_data['outputs_ts'], 
                                                      predict_data['outputs']):
        
        input_df = pd.DataFrame({'ds': inputs_ts, 
                                 'y': inputs})
        
        output_df = pd.DataFrame({'ds': outputs_ts, 
                                 'y': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['unique_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        # combined = combined[combined.y.isnull() == 0]
        # if len(combined) == 168:
        #         windows_pred.append(combined)
        # else: continue
        windows_pred.append(combined)
    
    windows_all_df = pd.concat(windows_all)
    windows_all_df.ds = pd.to_datetime(windows_all_df.ds)
    windows_pred_df = pd.concat(windows_pred)
    windows_pred_df.ds = pd.to_datetime(windows_pred_df.ds)
    windows_all_df['available_mask'] = windows_all_df.y.apply(lambda x: 0 if np.isnan(x) else 1)
    windows_pred_df['available_mask'] = windows_pred_df.y.apply(lambda x: 0 if np.isnan(x) else 1)

    res = forecast_building(windows_all_df, windows_pred_df)
    return res

In [ ]:
batch_size = 32
context_len = 168
horizon_len = 24

def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])

    # if df.shape[1] < 2:
    #     return None
        
    print(datetime.now(), df.shape, flush=True)

    res_all = []
    agg_metrics_all_patchtst = []
    agg_metrics_all_tft = []
    
    i = 0
    for building_name in df.columns:
        print(datetime.now(), i, '/', len(df.columns), building_name, flush=True)
        df1 = df[[building_name]]
        print(datetime.now(), i, '/', len(df.columns), building_name, df1.shape, flush=True)
        df1 = df1.loc[df1.first_valid_index():]
        print(datetime.now(), i, '/', len(df.columns), building_name, df1.shape, flush=True)
   
        res = process_building(df1)
        res['building'] = building_name
        res['filename'] = filename
        res_all.append(res)

        agg_metrics_patchtst = {}
        agg_metrics_patchtst['MSE'] = mse(res['y'], res['y_pred_PatchTST'])
        agg_metrics_patchtst['MAE'] = mae(res['y'], res['y_pred_PatchTST'])
        agg_metrics_patchtst['sMAPE'] = smape(res['y'], res['y_pred_PatchTST'])
        agg_metrics_patchtst['MAPE'] = mape(res['y'], res['y_pred_PatchTST'])
        agg_metrics_patchtst['RMSE'] = rmse(res['y'], res['y_pred_PatchTST'])
        agg_metrics_patchtst['NRMSE'] = rmse(res['y'], res['y_pred_PatchTST']) / res['y'].mean()
        agg_metrics_patchtst['QuantileLoss[0.5]'] = quantile_loss(res['y'], res['y_pred_PatchTST'], q=0.5)
        agg_metrics_patchtst_df = pd.DataFrame([agg_metrics_patchtst])
        agg_metrics_patchtst_df.insert(0, 'building', building_name)
        agg_metrics_patchtst_df.insert(0, 'filename', filename)
        agg_metrics_all_patchtst.append(agg_metrics_patchtst_df)

        agg_metrics_tft = {}
        agg_metrics_tft['MSE'] = mse(res['y'], res['y_pred_TFT'])
        agg_metrics_tft['MAE'] = mae(res['y'], res['y_pred_TFT'])
        agg_metrics_tft['sMAPE'] = smape(res['y'], res['y_pred_TFT'])
        agg_metrics_tft['MAPE'] = mape(res['y'], res['y_pred_TFT'])
        agg_metrics_tft['RMSE'] = rmse(res['y'], res['y_pred_TFT'])
        agg_metrics_tft['NRMSE'] = rmse(res['y'], res['y_pred_TFT']) / res['y'].mean()
        agg_metrics_tft['QuantileLoss[0.5]'] = quantile_loss(res['y'], res['y_pred_TFT'], q=0.5)
        agg_metrics_tft_df = pd.DataFrame([agg_metrics_tft])
        agg_metrics_tft_df.insert(0, 'building', building_name)
        agg_metrics_tft_df.insert(0, 'filename', filename)
        agg_metrics_all_tft.append(agg_metrics_tft_df)
        
        i += 1
        if i % 5 == 0:
            print(datetime.now(), 'Saving...')
            res_all_df = pd.concat(res_all).round(6)
            res_all_df.to_csv(f'forecasts/{dataset}/{os.path.basename(filename)}', index=False)                     

            agg_metrics_all_patchtst_df = pd.concat(agg_metrics_all_patchtst).round(6)            
            agg_metrics_all_patchtst_df.to_csv(f'results/{dataset}/PatchTST/agg_metrics_{os.path.basename(filename)}', index=False)

            agg_metrics_all_tft_df = pd.concat(agg_metrics_all_tft).round(6)            
            agg_metrics_all_tft_df.to_csv(f'results/{dataset}/TFT/agg_metrics_{os.path.basename(filename)}', index=False)
    
    
    res_all_df = pd.concat(res_all).round(6)
    res_all_df.to_csv(f'forecasts/{dataset}/{os.path.basename(filename)}', index=False)            

    agg_metrics_all_patchtst_df = pd.concat(agg_metrics_all_patchtst).round(6)            
    agg_metrics_all_patchtst_df.to_csv(f'results/{dataset}/PatchTST/agg_metrics_{os.path.basename(filename)}', index=False)

    agg_metrics_all_tft_df = pd.concat(agg_metrics_all_tft).round(6)            
    agg_metrics_all_tft_df.to_csv(f'results/{dataset}/TFT/agg_metrics_{os.path.basename(filename)}', index=False)

    return res_all_df, agg_metrics_all_patchtst_df, agg_metrics_all_tft_df

In [9]:
# torch.set_float32_matmul_precision('medium')
files_list = glob.glob('/home/user/New_Buildings_Datasets/Commercial/Enernoc/csv-only/processed/*.csv')

dataset = 'Enernoc'
os.makedirs(f'forecasts/{dataset}/', exist_ok = True)
os.makedirs(f'results/{dataset}/PatchTST/', exist_ok = True)
os.makedirs(f'results/{dataset}/TFT/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)
    # if results is not None:
    #     results.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)
    print('')

2024-11-22 16:03:16.949806 /home/user/New_Buildings_Datasets/Commercial/Enernoc/csv-only/processed/enernoc.csv
2024-11-22 16:03:17.071167 (8785, 100)
2024-11-22 16:03:17.071689 0 / 100 767
2024-11-22 16:03:17.072444 0 / 100 767 (8785, 1)
2024-11-22 16:03:17.073580 0 / 100 767 (8785, 1)


Seed set to 1
Seed set to 1
/home/user/miniconda3/envs/neural_forecast/lib/python3.9/site-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type              | Params | Mode 
-----------------------------------------------------------
0 | loss         | DistributionLoss  | 5      | train
1 | padder_train | ConstantPad1d     | 0      | train
2 | scaler       | 

Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Predicting: |                                                                                                 …

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                                                                 …

/home/user/miniconda3/envs/neural_forecast/lib/python3.9/site-packages/neuralforecast/core.py:209: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


2024-11-22 17:42:04.091181 Saving...

